## Tracking daily changes in historic Cocoa ICE (CC) futures 
In this notebook we are extracting historic daily close prices for any cocoa futures,using continuation Rics up to 12 months forward . In addition, we are analysing the spread between two contracts.

### Import required packages

In [ ]:
import pandas as pd
import refinitiv.data as rd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

rd.open_session()

### Main code and functions

In [ ]:
# Function to generate the ric list

def gen_contracts(baseString):
    contracts = []
    for i in range(1, 37):
        contracts.append(baseString+'c'+str(i))
    return contracts
# Function to retrieve the data


def get_contracts_data(baseCon, start_date, end_date):
    contracts = gen_contracts(baseCon)
    print(contracts)
    df = rd.get_history(contracts, fields=[
                        "MID_PRICE"], start=start_date, end=end_date, interval='daily', count=10000)
    return df

# Function for anlyzing spread


def spread_analysis(df, c1, c2):
    df = df.reset_index().ffill()
    df['spread'] = df[c1] - df[c2]
    df['Z_score_spread'] = (df.spread - df.spread.mean())/df.spread.std(ddof=0)

    fig = px.line(df, x='Date', y='Z_score_spread',
                  title='Normalized Spread b/w two contracts')
    fig.add_shape(type='line',
                  x0=df['Date'].iloc[0],
                  y0=-3,
                  x1=df['Date'].iloc[-1],
                  y1=-3,
                  line=dict(color='Red',),
                  xref='x',
                  yref='y'
                  )
    fig.add_shape(type='line',
                  x0=df['Date'].iloc[0],
                  y0=3,
                  x1=df['Date'].iloc[-1],
                  y1=3,
                  line=dict(color='Red',),
                  xref='x',
                  yref='y'

                  )
    fig.show()

### Enter RIC root to get data. 
Enter the Ric root for each metal to get data, for example Cc1 for December-2024 and Cc2 for March-2025

In [ ]:
from refinitiv_widgets import TextField
from refinitiv_widgets import DatetimePicker
baseCon = TextField(placeholder='Enter Ric Root')

baseCon

In [ ]:
from refinitiv_widgets import TextField
dp = DatetimePicker(
    range_mode=True,
    duplex_mode='split',
    weekdays_only=True,
    value=["2022-01-01", "2022-09-09"])
dp

### Extract Data
File will be saved in user home directory. (same level with Examples folder)

In [ ]:
from pathlib import Path
rd.open_session()
df = get_contracts_data(baseCon.value, dp.value[0], dp.value[1])
fileName = baseCon.value + '.csv'
df.to_csv(Path.home() / fileName)
rd.close_session()
df.head()

### Spread Analysis
Enter two contratcs from above table (for example Cc1,Cc2) to analyze the spread between them.

In [ ]:
tf = TextField(placeholder='input two contracts separted by comma')
tf

In [ ]:
spread_analysis(df, tf.value.split(',')[0], tf.value.split(',')[1])

### Spread Analysis Different Contracts

In [ ]:
rd.open_session()
start_date = dp.value[0]
end_date = dp.value[1]
contracts = "COCO.L"
df_coco = rd.get_history(contracts, fields=[
                    "MID_PRICE"], start=start_date, end=end_date, interval='daily', count=10000)
df_coco.head(5)
df["COCO.L"] = df_coco["MID_PRICE"]
tf_cc = TextField(placeholder='input one contract')
tf_cc

In [ ]:
spread_analysis(df, tf_cc.value, "COCO.L")